In [1]:
import logging
import time
import xarray as xr
import glob
import pandas as pd

from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
from examples.download_kernels import download_kernels
from src.config import ShSpOpt
from src.shadowspy.data_handling import fetch_and_process_data
from src.shadowspy.dem_processing import prepare_dem_mesh
from src.shadowspy.helpers import setup_directories, process_data_list
from src.shadowspy.raster_products import basic_raster_stats
from src.shadowspy.utilities import run_log

Setup config and directories

In [3]:
opt = ShSpOpt()
opt.setup_config(config_file='MoonSP_render_config.yaml')
opt.set(siteid='MoonSPsmall')
opt.display()

setup_directories(opt)

### ShSpOpt.siteid updated to MoonSPsmall.
initialized = True
siteid = MoonSPsmall
Fsun = 1361
flux_path = None
wavelength = None
Rb = 1737.4
base_resolution = 120
config_file = None
download_kernels = False
irradiance_only = False
max_extension = 200e3
extres = {'20e3': 40, '60e3': 120, '100e3': 240, '300e3': 480}
lonlat0_stereo = [0, -90]
bbox_roi = ['50e3', '-150e3', '150e3', '-50e3']
shapefile_roi = None
mesh_ext = .vtk
epos_utc = []
start_time = 2026-01-30 15:00:00.0
end_time = 2026-01-30 17:00:00.0
time_step_hours = 1
azi_ele_path = None
images_index = /home/sberton2/Lavoro/projects/validate_shadowspy/final_selection_0_sel0.csv
point_source = True
source = SUN
extsource_coord = ./aux/coordflux_100pts_outline33_centerlast_R1_F1_stdlimbdark.txt
root = ./
indir = ./aux/
dem_path = /home/sberton2/Lavoro/projects/validate_shadowspy/LDEM_80S_40MPP_ADJ.TIF
fartopo_path = same_dem
outdir = ./out/
tmpdir = ./tmp/


download kernels

In [ ]:
if opt.download_kernels:
    download_kernels()

prepare mesh of the input dem

In [4]:
start = time.time()
logging.info(f"- Computing trimesh for {opt.dem_path}...")
inner_mesh_path, outer_mesh_path, dem_path = prepare_dem_mesh(opt.dem_path, opt.tmpdir, opt.siteid, opt)
logging.info(f"- Meshes generated after {round(time.time() - start, 2)} seconds.")

INFO:root:- Computing trimesh for /home/sberton2/Lavoro/projects/validate_shadowspy/LDEM_80S_40MPP_ADJ.TIF...
INFO:root:Clipped /home/sberton2/Lavoro/projects/validate_shadowspy/LDEM_80S_40MPP_ADJ.TIF to 50e3_-150e3_150e3_-50e3 and saved to ./tmp/clipped_dem_MoonSPsmall_50e3_-150e3_150e3_-50e3.tif


- Delauney mesh computed and saved to ./tmp/MoonSPsmall_b120_dn1_st.vtk.


INFO:root:- Reading existing stacked mesh file
INFO:root:- Meshes generated after 11.45 seconds.


- Delauney mesh computed and saved to ./tmp/MoonSPsmall_b120_dn1.vtk.


Determine the mode and prepare data list

In [5]:
data_list, use_azi_ele, use_image_times = fetch_and_process_data(opt)
print(f"- Illuminating input DEM at {data_list}.")

- Illuminating input DEM at [('M142497921RE', ' 2010-10-23 18:30:53.712', '/home/sberton2/Lavoro/projects/validate_shadowspy/final_selection_map/M142497921RE_map.tif'), ('M142504702LE', ' 2010-10-23 20:23:55.473', '/home/sberton2/Lavoro/projects/validate_shadowspy/final_selection_map/M142504702LE_map.tif'), ('M1108619922RE', ' 2012-11-27 01:44:15.386', '/home/sberton2/Lavoro/projects/validate_shadowspy/final_selection_map/M1108619922RE_map.tif')].


/home/sberton2/Lavoro/code/shadowspy/src/shadowspy/image_util.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumindex_red.PRODUCT_ID = cumindex_red.PRODUCT_ID.str.strip()
/home/sberton2/Lavoro/code/shadowspy/src/shadowspy/data_handling.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_list = [(row[0], row[1], row[2]) for idx, row in data_list.iterrows()]


Common arguments for both cases

In [6]:
common_args = {
    'meshes': {'stereo': f"{inner_mesh_path}_st{opt.mesh_ext}", 'cart': f"{inner_mesh_path}{opt.mesh_ext}"},
    'basemesh_path': outer_mesh_path + opt.mesh_ext,
    'path_to_furnsh': f"{opt.indir}simple.furnsh",
    'point': opt.point_source,
    'extsource_coord': opt.extsource_coord,
    'source': opt.source,
    'dem_path': dem_path,
    'crs': 'PROJCS["Moon (2015) - Sphere / Ocentric / South Polar", BASEGEOGCRS["Moon (2015) - Sphere / Ocentric", DATUM["Moon (2015) - Sphere", ELLIPSOID["Moon (2015) - Sphere",1737400,0, LENGTHUNIT["metre",1]]], PRIMEM["Reference Meridian",0, ANGLEUNIT["degree",0.0174532925199433]], ID["IAU",30100,2015]], CONVERSION["South Polar", METHOD["Polar Stereographic (variant A)", ID["EPSG",9810]], PARAMETER["Latitude of natural origin",-90, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8801]], PARAMETER["Longitude of natural origin",0, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8802]], PARAMETER["Scale factor at natural origin",1, SCALEUNIT["unity",1], ID["EPSG",8805]], PARAMETER["False easting",0, LENGTHUNIT["metre",1], ID["EPSG",8806]], PARAMETER["False northing",0, LENGTHUNIT["metre",1], ID["EPSG",8807]]], CS[Cartesian,2], AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1]], AXIS["(N)",north, ORDER[2], LENGTHUNIT["metre",1]], ID["IAU",30135,2015]]'
}

actually compute irradiance at each element of data_list

In [7]:
dsi_epo_path_dict = process_data_list(data_list, common_args, use_azi_ele, use_image_times, opt)

  0%|          | 0/3 [00:00<?, ?it/s]WARNING:root:* Outer mesh not yet implemented for render_match_image. Setting back to None.


- Processing M142497921RE and clipping to /home/sberton2/Lavoro/projects/validate_shadowspy/final_selection_map/M142497921RE_map.tif...
- Cropping DEM to                                             geometry
0  POLYGON ((-112.474 154.752, -112.492 154.775, ...


/home/sberton2/Lavoro/code/shadowspy/src/shadowspy/render_dem.py:210: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  crop_mesh(dem_mask, meshes, mask=dem_mask, meshes_cropped=meshes_cropped)
/home/sberton2/Lavoro/code/shadowspy/shadow-env/lib/python3.10/site-packages/geopandas/geodataframe.py:2195: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: PROJCS["Moon (2015) - Sphere / Ocentric / South Po ...

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026
ERROR:root:* Meshes and mask do not overlap. Weird. Stop
  0%|          | 0/3 [00:03<?, ?it/s]


NameError: name 'exit' is not defined

In [ ]:
render_paths = glob.glob(f"{opt.outdir}{opt.siteid}/{opt.siteid}_*.tif")[:]
pd.DataFrame.from_records([dsi_epo_path_dict]).T.to_csv(f'{opt.outdir}{opt.siteid}/dsi_epo_paths.csv')

fig, axes = plt.subplots(nrows=len(render_paths), ncols=1)
for idx, render in tqdm(enumerate(render_paths)):
    da = xr.open_dataarray(render)
    # da = da.sel(band=1).coarsen({'x': 3, 'y': 3}, boundary='trim').mean() 
    da.plot(robust=True, cmap='Greys', ax=axes[idx])

plt.show()

prepare mean, sum, max stats rasters and set-up log file

In [ ]:
# from datetime import datetime
dsi_epo_path = pd.read_csv(f'{opt.outdir}{opt.siteid}/dsi_epo_paths.csv').set_index('Unnamed: 0').to_dict()['0']
# dsi_epo_path = {datetime.strptime(str(k), '%Y%m%d%H%M%S'):v for k,v in dsi_epo_path.items()}
dem = xr.open_dataarray(common_args['dem_path'])
basic_raster_stats(dsi_epo_path, opt.time_step_hours, crs=dem.rio.crs, outdir=opt.outdir, siteid=opt.siteid)

In [ ]:
run_log(Fsun=opt.Fsun, Rb=opt.Rb, base_resolution=opt.base_resolution, siteid=opt.siteid, dem_path=dem_path, outdir=opt.outdir,
        start_time=opt.start_time, end_time=opt.end_time, time_step_hours=opt.time_step_hours,
        runtime_sec=round(time.time() - start, 2), logpath=f"{opt.outdir}{opt.siteid}/illum_stats_{opt.siteid}_{int(time.time())}.json")
logging.info(f"Completed in {round(time.time() - start, 2)} seconds.")